In [153]:
import torch 
import torch.nn as nn 
from torch.optim import SGD

from torch.utils.data import Dataset,DataLoader,random_split
from torch.nn.functional import one_hot

## Other modules 

import numpy as np 

import os 
import pandas as pd 

In [154]:
## defining a custom dataset 

class WineDataset(Dataset):

    def __init__(self,transform=None):

        super(WineDataset,self).__init__()
        xy = np.loadtxt('./data/wine/wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        X,y=xy[:,1:],xy[:,0]# y will be reshape .reshape(-1,1) when use bracket
        self.num_class=np.unique(y).shape[0]
        self.num_samples=xy.shape[0]
        self.x_data=torch.from_numpy(X)
        self.y_data=torch.from_numpy(y).to(torch.int64)-1 ## dont accept int32 for some reasons 

        print('Num of Class: {}'.format(self.num_class))

    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, index):
        x=self.x_data[index]
        #y=one_hot(self.y_data[index],self.num_class); In pytorch nn.CrossEntropyLoss automatically handles raw dataset without hot-encodding
        return x,self.y_data[index]
        


In [155]:
dataset=WineDataset()
x,y=dataset[:3]
print(x.shape)
print(y.shape)

Num of Class: 3
torch.Size([3, 13])
torch.Size([3])


In [167]:
torch.manual_seed(101)

total_size=len(dataset)
train_size=int(0.7*total_size)
val_size=int(0.15*total_size)
test_size=total_size-train_size-val_size

train_dataset,val_dataset,test_dataset=random_split(dataset=dataset,lengths=[train_size,val_size,test_size]) 

In [168]:
train_dataloader=DataLoader(dataset=train_dataset,
                            batch_size=32,
                            shuffle=True,
                            num_workers=2)

val_dataloader=DataLoader(dataset=val_dataset,
                            batch_size=32,
                            shuffle=True,
                            num_workers=2)

test_dataloader=DataLoader(dataset=test_dataset,
                            batch_size=32,
                            shuffle=True,
                            num_workers=2)

In [158]:
## The above dataloader is in the iterator form. 

dataiter=iter(train_dataloader)
feat,lbs=next(dataiter)
print(feat.shape) ## batch_size,
print(lbs.shape) ## batch_size,num_labels

torch.Size([32, 13])
torch.Size([32])


In [159]:
num_features=13
num_class=3

In [160]:
class WineClassModel(nn.Module):

    def __init__(self):
        super(WineClassModel,self).__init__()
        self.nn=nn.Sequential(nn.Linear(13,256),
                              nn.ReLU(),
                              nn.Dropout(0.5),
                              nn.Linear(256,128),
                              nn.ReLU(),
                              nn.Dropout(0.5),
                              nn.Linear(128,3)
        )
        
    def forward(self,x):
        return self.nn(x)

In [161]:
model=WineClassModel()

In [162]:
num_epochs=100
criterion=nn.CrossEntropyLoss()
opt=SGD(model.parameters(),lr=0.001)

In [178]:
# checking the model

model(x).shape[0]

3

In [171]:
a=torch.tensor([1,2,3,4])
b=torch.tensor([1,2,5,7])

In [185]:
torch.tensor(a==b,dtype=torch.float16).sum().item()

/tmp/ipykernel_306335/896790710.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(a==b,dtype=torch.float16).sum().item()


2.0

In [165]:
for epoch in range(num_epochs):

    for step_num,(features,labels) in enumerate(train_dataloader):
        pred_labels=model(features)
        loss=criterion(pred_labels,labels)
        loss.backward()
        opt.step()
        opt.zero_grad()

        if step_num%32==0:

            print('Epoch: {}/{} | Step: {}/{}'.format(epoch,num_epochs,step_num,32))
            print('loss: {:.3f}'.format(loss.item()))


Epoch: 0/100 | Step: 0/32
loss: 1.159
Epoch: 1/100 | Step: 0/32
loss: 1.073
Epoch: 2/100 | Step: 0/32
loss: 1.027
Epoch: 3/100 | Step: 0/32
loss: 1.081
Epoch: 4/100 | Step: 0/32
loss: 1.095
Epoch: 5/100 | Step: 0/32
loss: 1.078
Epoch: 6/100 | Step: 0/32
loss: 1.073
Epoch: 7/100 | Step: 0/32
loss: 1.087
Epoch: 8/100 | Step: 0/32
loss: 1.223
Epoch: 9/100 | Step: 0/32
loss: 1.102
Epoch: 10/100 | Step: 0/32
loss: 1.163
Epoch: 11/100 | Step: 0/32
loss: 1.107
Epoch: 12/100 | Step: 0/32
loss: 1.020
Epoch: 13/100 | Step: 0/32
loss: 1.096
Epoch: 14/100 | Step: 0/32
loss: 1.063
Epoch: 15/100 | Step: 0/32
loss: 1.086
Epoch: 16/100 | Step: 0/32
loss: 1.113
Epoch: 17/100 | Step: 0/32
loss: 0.994
Epoch: 18/100 | Step: 0/32
loss: 1.195
Epoch: 19/100 | Step: 0/32
loss: 1.024
Epoch: 20/100 | Step: 0/32
loss: 1.115
Epoch: 21/100 | Step: 0/32
loss: 1.057
Epoch: 22/100 | Step: 0/32
loss: 1.074
Epoch: 23/100 | Step: 0/32
loss: 1.046
Epoch: 24/100 | Step: 0/32
loss: 1.120
Epoch: 25/100 | Step: 0/32
loss: 1.

In [166]:
##Now for evaluation set to model.eval()

model.eval()

WineClassModel(
  (nn): Sequential(
    (0): Linear(in_features=13, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=128, out_features=3, bias=True)
  )
)

In [195]:
with torch.no_grad():
    accuracies=[]

    for test_features,test_labels in test_dataloader:
        pred_labels=model(test_features)
        pred_labels=torch.argmax(pred_labels,1)
        true_prediction=torch.tensor(pred_labels==test_labels,dtype=torch.float32).sum()
        total_batch_size=torch.tensor(pred_labels.shape[0],dtype=torch.float32)
        accuracies.append(torch.div(true_prediction,total_batch_size).item())
        print(accuracies)


print('Average Accuracy: {:.2f}'.format(np.mean(accuracies)))

[0.3571428656578064]
Average Accuracy: 0.36


/tmp/ipykernel_306335/3893402697.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  true_prediction=torch.tensor(pred_labels==test_labels,dtype=torch.float32).sum()


In [192]:
torch.div(10,2)

tensor(5.)

In [193]:
accuracies

[2.799999952316284]